In [1]:
import re
from pprint import pprint
import pandas as pd

In [2]:
cd ~/googledrive/maca_uploads/

/Users/olgabot/Google Drive/MACA uploads


In [16]:
!sed 's/tissue/tiss/g' Lung/backup/MACA_Plate_Notebook.Rmd > Lung/MACA_Plate_Notebook_tiss_not_tissue.Rmd

In [17]:
rmds = '*/*Plate*Rmd'
!ls $rmds

Aorta/MACA_Plate_Notebook_Aorta.Rmd
Bladder/MACA_Plate_Notebook.new3mo.Rmd
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd
Colon/MACA_Plate_Notebook_Colon_102717.Rmd
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd
Fat/MACA_Plate_Notebook_All_Fat.Rmd
Heart/MACA_Plate_Notebook_HeartSample.Rmd
Kidney/Kidney_MACA_Plate_Notebook.Rmd
Liver/MACA_Plate-liver-3m.Rmd
Lung/MACA_Plate_Notebook_tiss_not_tissue.Rmd
Mammary_Gland/MACA_Plate_Notebook_mammary.Rmd
Muscle/MACA_Plate_Notebook.Rmd
Pancreas/Pancreas_MACA_Plate_Notebook.Rmd
Skin/MACA_Plate_Notebook_All_Skin.Rmd
Spleen/Spleen_MACA_Plate_Notebook.Rmd
Thymus/MACA_Plate_Notebook_Thymus final3_Nov13.Rmd
Tongue/MACA_3mo_Tongue_Plate_Notebook.Rmd
Trachea/MACA_Plate_Notebook.Rmd


## Parse CreateSeuratObject and create parameter dataframe

In [18]:
!grep -A 1 CreateSeuratObject  $rmds



--
Bladder/MACA_Plate_Notebook.new3mo.Rmd:tiss <- CreateSeuratObject(raw.data = raw.data, project = tissue_of_interest, 
Bladder/MACA_Plate_Notebook.new3mo.Rmd-                    min.cells = 5, min.genes = 5)
--
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:tiss <- CreateSeuratObject(raw.data = raw.data, project = tissue_of_interest, 
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd-                    min.cells = 5, min.genes = 5)
--
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:tiss <- CreateSeuratObject(raw.data = raw.data, project = tissue_of_interest, 
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd-                    min.cells = 5, min.genes = 5)
--
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:tiss <- CreateSeuratObject(raw.data = raw.data, project = tissue_of_interest, 
Colon/MACA_Plate_Notebook_Colon_102717.Rmd-                    min.cells = 5, min.genes = 5)
--
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd:tiss <- CreateSeuratObject(raw.data = raw.data, projec

Some different parameters for min cells and min genes

In [19]:
import re

matches = !grep -A 1 CreateSeuratObject  $rmds
pprint(matches[:5])

index = ['CreateSeuratObject_min.cells', 'CreateSeuratObject_min.genes']

seriess = []

for line in matches:
    if 'min.cells' in line:
        tissue = line.split('/')[0]
        min_cells = int(re.findall('min\.cells = (\d+)', line)[0])
        min_genes = int(re.findall('min\.genes = (\d+)', line)[0])
        series = pd.Series([min_cells, min_genes], name=tissue, index=index)
        seriess.append(series)
        
parameters = pd.concat(seriess, axis=1).T
parameters

['Aorta/MACA_Plate_Notebook_Aorta.Rmd:tiss <- CreateSeuratObject(raw.data = '
 'raw.data, project = tissue_of_interest, ',
 'Aorta/MACA_Plate_Notebook_Aorta.Rmd-                    min.cells = 5, '
 'min.genes = 5)',
 '--',
 'Bladder/MACA_Plate_Notebook.new3mo.Rmd:tiss <- CreateSeuratObject(raw.data = '
 'raw.data, project = tissue_of_interest, ',
 'Bladder/MACA_Plate_Notebook.new3mo.Rmd-                    min.cells = 5, '
 'min.genes = 5)']


,CreateSeuratObject_min.cells,CreateSeuratObject_min.genes
Aorta,5,5
Bladder,5,5
Brain_FACS_microglia,5,5
Brain_FACS_neurons,5,5
Colon,5,5
Diaphragm,5,5
Fat,5,5
Heart,5,5
Kidney,5,5
Liver,5,5


## Add parameters from FilterCells

In [20]:
!grep -A 1 FilterCells $rmds



--
Bladder/MACA_Plate_Notebook.new3mo.Rmd:tiss <- FilterCells(object = tiss, subset.names = c("nGene", "nReads"), 
Bladder/MACA_Plate_Notebook.new3mo.Rmd-    low.thresholds = c(500, 50000), high.thresholds = c(25000, 2000000))
--
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:tiss <- FilterCells(object = tiss, subset.names = c("nGene", "nReads"), 
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd-    low.thresholds = c(500, 50000), high.thresholds = c(25000, 2000000))
--
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:tiss <- FilterCells(object = tiss, subset.names = c("nGene", "nReads"), 
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd-    low.thresholds = c(500, 50000), high.thresholds = c(25000, 2000000))
--
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:tiss <- FilterCells(object = tiss, subset.names = c("nGene", "nReads"), 
Colon/MACA_Plate_Notebook_Colon_102717.Rmd-    low.thresholds = c(500, 50000), high.thresholds = c(25000, 2000000))
--
Diaphragm/MACA_Plate_Notebook_Dia

A few different parameters for min/max genes, min/max cells

In [21]:
columns = ['FilterCells_min.genes', 'FilterCells_min.reads', 'FilterCells_max.genes', 'FilterCells_max.reads']

for col in columns:
    parameters[col] = None

In [22]:
seriess = []

matches = !grep -A 1 FilterCells $rmds
matches[:5]

for line in matches:
    if 'low.thresholds' in line:
        tissue = line.split('/')[0]
        min_genes, min_reads = map(int, re.findall('low.thresholds = c\((\d+), (\d+)\)', line)[0])
        max_genes, max_reads = map(int, re.findall('high.thresholds = c\((\d+), (\d+)\)', line)[0])
        parameters.loc[tissue, columns] = min_genes, min_reads, max_genes, max_reads
parameters

,CreateSeuratObject_min.cells,CreateSeuratObject_min.genes,FilterCells_min.genes,FilterCells_min.reads,FilterCells_max.genes,FilterCells_max.reads
Aorta,5,5,500,50000,25000,2000000
Bladder,5,5,500,50000,25000,2000000
Brain_FACS_microglia,5,5,500,50000,25000,2000000
Brain_FACS_neurons,5,5,500,50000,25000,2000000
Colon,5,5,500,50000,25000,2000000
Diaphragm,5,5,500,50000,25000,2000000
Fat,5,5,500,50000,25000,2000000
Heart,5,5,500,50000,25000,2000000
Kidney,5,5,500,50000,25000,2000000
Liver,5,5,500,50000,25000,5000000


## Check `NormalizeData`, `ScaleData`, and `FindVariableGenes` parameters

In [23]:
! grep -A 2 -E '^tiss.+NormalizeData' $rmds




--
Bladder/MACA_Plate_Notebook.new3mo.Rmd:tiss <- NormalizeData(object = tiss)
Bladder/MACA_Plate_Notebook.new3mo.Rmd-tiss <- ScaleData(object = tiss, vars.to.regress = c("nReads", "percent.ribo","Rn45s"))
Bladder/MACA_Plate_Notebook.new3mo.Rmd-tiss <- FindVariableGenes(object = tiss, do.plot = TRUE, x.high.cutoff = Inf, y.cutoff = 0.5)
--
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:tiss <- NormalizeData(object = tiss)
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd-tiss <- ScaleData(object = tiss, vars.to.regress = c("nReads", "percent.ribo","Rn45s"))
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd-tiss <- FindVariableGenes(object = tiss, do.plot = TRUE, x.high.cutoff = Inf, y.cutoff = 0.5)
--
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:tiss <- NormalizeData(object = tiss)
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd-tiss <- ScaleData(object = tiss, vars.to.regress = c("nReads", "percent.ribo","Rn45s"))
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd-tiss 

Everyone used the same parameters for `NormalizeData`, `ScaleData`, and `FindVariableGenes`

## Check RunPCA parameters

In [24]:
! grep -E '^tiss.+RunPCA' $rmds


Bladder/MACA_Plate_Notebook.new3mo.Rmd:tiss <- RunPCA(object = tiss, do.print = FALSE)
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:tiss <- RunPCA(object = tiss, do.print = FALSE)
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:tiss <- RunPCA(object = tiss, do.print = FALSE)
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:tiss <- RunPCA(object = tiss, do.print = FALSE)
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd:tiss <- RunPCA(object = tiss, pc.genes = tiss@var.genes, do.print = TRUE, pcs.print = 1:5, 

Heart/MACA_Plate_Notebook_HeartSample.Rmd:tiss <- RunPCA(object = tiss, do.print = FALSE)

Liver/MACA_Plate-liver-3m.Rmd:tiss <- RunPCA(object = tiss, do.print = FALSE, pcs.compute=30)
Lung/MACA_Plate_Notebook_tiss_not_tissue.Rmd:tiss <- RunPCA(object = tiss, pc.genes = tiss@var.genes, do.print = TRUE, pcs.print = 1:5, genes.print = 5, pcs.compute = 30)
Mammary_Gland/MACA_Plate_Notebook_mammary.Rmd:tiss <- RunPCA(object = tiss, do.print = FALSE)
Muscle/MACA_Plate_Notebook.Rmd:tis

Everyone used the same parameters for `RunPCA` which computes 20 PCs by default.

## Get number of principal components for each tissue

In [25]:
! grep ^n.pcs $rmds


Bladder/MACA_Plate_Notebook.new3mo.Rmd:n.pcs = 10
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:n.pcs = 10
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:n.pcs = 15
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:n.pcs = 20
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd:n.pcs = 10

Heart/MACA_Plate_Notebook_HeartSample.Rmd:n.pcs = 10

Liver/MACA_Plate-liver-3m.Rmd:n.pcs = 10
Lung/MACA_Plate_Notebook_tiss_not_tissue.Rmd:n.pcs = 20
Mammary_Gland/MACA_Plate_Notebook_mammary.Rmd:n.pcs = 10
Muscle/MACA_Plate_Notebook.Rmd:n.pcs = 15
Pancreas/Pancreas_MACA_Plate_Notebook.Rmd:n.pcs = 11



Tongue/MACA_3mo_Tongue_Plate_Notebook.Rmd:n.pcs = 12
Trachea/MACA_Plate_Notebook.Rmd:n.pcs = 12


In [26]:
matches = ! grep ^n.pcs $rmds

column = 'n.pcs'

for line in matches:
    tissue = line.split('/')[0]
    print('---', tissue, '---')

    n_pcs = int(re.findall('n\.pcs = (\d+)', line)[0])
    parameters.loc[tissue, column] = n_pcs
    
parameters[column] = parameters[column].astype(int)
parameters

--- Aorta ---
--- Bladder ---
--- Brain_FACS_microglia ---
--- Brain_FACS_neurons ---
--- Colon ---
--- Diaphragm ---
--- Fat ---
--- Heart ---
--- Kidney ---
--- Liver ---
--- Lung ---
--- Mammary_Gland ---
--- Muscle ---
--- Pancreas ---
--- Skin ---
--- Spleen ---
--- Thymus ---
--- Tongue ---
--- Trachea ---


,CreateSeuratObject_min.cells,CreateSeuratObject_min.genes,FilterCells_min.genes,FilterCells_min.reads,FilterCells_max.genes,FilterCells_max.reads,n.pcs
Aorta,5,5,500,50000,25000,2000000,10
Bladder,5,5,500,50000,25000,2000000,10
Brain_FACS_microglia,5,5,500,50000,25000,2000000,10
Brain_FACS_neurons,5,5,500,50000,25000,2000000,15
Colon,5,5,500,50000,25000,2000000,20
Diaphragm,5,5,500,50000,25000,2000000,10
Fat,5,5,500,50000,25000,2000000,20
Heart,5,5,500,50000,25000,2000000,10
Kidney,5,5,500,50000,25000,2000000,9
Liver,5,5,500,50000,25000,5000000,10


## Get clustering resolution

In [27]:
! grep --line-number -A 1 -E '^tiss.+FindClusters' $rmds



--
Bladder/MACA_Plate_Notebook.new3mo.Rmd:207:tiss <- FindClusters(object = tiss, reduction.type = "pca", dims.use = 1:n.pcs, 
Bladder/MACA_Plate_Notebook.new3mo.Rmd-208-    resolution = res.used, print.output = 0, save.SNN = TRUE)
--
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:149:tiss <- FindClusters(object = tiss, reduction.type = "pca", dims.use = 1:n.pcs, 
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd-150-    resolution = res.used, print.output = 0, save.SNN = TRUE)
--
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:146:tiss <- FindClusters(object = tiss, reduction.type = "pca", dims.use = 1:n.pcs, 
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd-147-    resolution = res.used, print.output = 0, save.SNN = TRUE)
--
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:144:tiss <- FindClusters(object = tiss, reduction.type = "pca", dims.use = 1:n.pcs, 
Colon/MACA_Plate_Notebook_Colon_102717.Rmd-145-    resolution = res.used, print.output = 0, save.SNN = TRUE)
--
Diaphragm/MAC

In [28]:
! grep --line-number -E '(^res.used)|(resolution = )' $rmds




Bladder/MACA_Plate_Notebook.new3mo.Rmd:205:res.used <- 1
Bladder/MACA_Plate_Notebook.new3mo.Rmd:208:    resolution = res.used, print.output = 0, save.SNN = TRUE)
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:147:res.used <- 0.5
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:150:    resolution = res.used, print.output = 0, save.SNN = TRUE)
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:144:res.used <- 0.5
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:147:    resolution = res.used, print.output = 0, save.SNN = TRUE)
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:347:    resolution = sub.res.used, print.output = 0, save.SNN = TRUE)
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:142:res.used <- 0.5
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:145:    resolution = res.used, print.output = 0, save.SNN = TRUE)
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:321:    resolution = sub.res.used, print.output = 0, save.SNN = TRUE)
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd:150:    resoluti

Different resolutions used, and Diaphragm specified in a different way.

In [29]:
matches = ! grep --line-number -E '(^res.used)|(resolution = )' $rmds

column = 'FindClusters_res.used'

resolution_tissues = 'Diaphragm', 'Lung'

seen_tissues = set([])

for line in matches:
    tissue = line.split('/')[0]
    if tissue in seen_tissues:
        continue

    seen_tissues.add(tissue)
    if tissue in resolution_tissues:
        print(line)
        # Thankfully Diaphragm used the same resolution for "tiss" and "sub.tiss"
        value = float(re.findall('resolution = (\d+\.?\d?)', line)[0])
        parameters.loc[tissue, column] = value
    else:
        if 'res.used <-' in line:
            value = float(re.findall('res\.used <- (\d+\.?\d?)', line)[0])
            parameters.loc[tissue, column] = value
    
parameters

Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd:150:    resolution = 1, print.output = 0, save.SNN = TRUE, force.recalc = TRUE)
Lung/MACA_Plate_Notebook_tiss_not_tissue.Rmd:136:tiss <- FindClusters(object = tiss, reduction.type = "pca", dims.use = 1:n.pcs, resolution = 3, print.output = 0, save.SNN = TRUE, force.recalc = TRUE)


,CreateSeuratObject_min.cells,CreateSeuratObject_min.genes,FilterCells_min.genes,FilterCells_min.reads,FilterCells_max.genes,FilterCells_max.reads,n.pcs,FindClusters_res.used
Aorta,5,5,500,50000,25000,2000000,10,0.5
Bladder,5,5,500,50000,25000,2000000,10,1.0
Brain_FACS_microglia,5,5,500,50000,25000,2000000,10,0.5
Brain_FACS_neurons,5,5,500,50000,25000,2000000,15,0.5
Colon,5,5,500,50000,25000,2000000,20,0.5
Diaphragm,5,5,500,50000,25000,2000000,10,1.0
Fat,5,5,500,50000,25000,2000000,20,0.5
Heart,5,5,500,50000,25000,2000000,10,0.5
Kidney,5,5,500,50000,25000,2000000,9,2.5
Liver,5,5,500,50000,25000,5000000,10,0.5


## Check TSNE parameters

In [30]:
! grep --line-number -E '^tiss.+RunTSNE' $rmds


Bladder/MACA_Plate_Notebook.new3mo.Rmd:214:tiss <- RunTSNE(object = tiss, dims.use = 1:n.pcs, seed.use = 10, perplexity=30)
Brain_FACS_microglia/MACA_Plate_Notebook_microglia_v2.Rmd:156:tiss <- RunTSNE(object = tiss, dims.use = 1:n.pcs, seed.use = 10, perplexity=30)
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:154:tiss <- RunTSNE(object = tiss, dims.use = 1:n.pcs, seed.use = 10, perplexity=30)
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:151:tiss <- RunTSNE(object = tiss, dims.use = 1:n.pcs, seed.use = 10, perplexity=30)
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd:155:tiss <- RunTSNE(object = tiss, dims.use = 1:n.pcs, seed.use = 10, check_duplicates = F, perplexity=30)

Heart/MACA_Plate_Notebook_HeartSample.Rmd:152:tiss <- RunTSNE(object = tiss, dims.use = 1:n.pcs, seed.use = 10, perplexity=30)

Liver/MACA_Plate-liver-3m.Rmd:157:tiss <- RunTSNE(object = tiss, dims.use = 1:n.pcs, seed.use = 10, perplexity=30)
Lung/MACA_Plate_Notebook_tiss_not_tissue.Rmd:141:tiss <- RunTSNE(object = tiss

Differet parameters for `perplexity`, and can reuse `n.pcs` for `dims.use`

In [31]:
matches = ! grep --line-number -E '^tiss.+RunTSNE' $rmds

column = 'RunTSNE_perplexity'

for line in matches:
    tissue = line.split('/')[0]
    if 'perplexity' in line:
        value = int(re.findall('perplexity=(\d+)', line)[0])
    else:
        # If they didn't set it, then the default value is 30
        value = 30
    parameters.loc[tissue, column] = value
    
parameters[column] = parameters[column].astype(int)
parameters

,CreateSeuratObject_min.cells,CreateSeuratObject_min.genes,FilterCells_min.genes,FilterCells_min.reads,FilterCells_max.genes,FilterCells_max.reads,n.pcs,FindClusters_res.used,RunTSNE_perplexity
Aorta,5,5,500,50000,25000,2000000,10,0.5,30
Bladder,5,5,500,50000,25000,2000000,10,1.0,30
Brain_FACS_microglia,5,5,500,50000,25000,2000000,10,0.5,30
Brain_FACS_neurons,5,5,500,50000,25000,2000000,15,0.5,30
Colon,5,5,500,50000,25000,2000000,20,0.5,30
Diaphragm,5,5,500,50000,25000,2000000,10,1.0,30
Fat,5,5,500,50000,25000,2000000,20,0.5,30
Heart,5,5,500,50000,25000,2000000,10,0.5,30
Kidney,5,5,500,50000,25000,2000000,9,2.5,30
Liver,5,5,500,50000,25000,5000000,10,0.5,30


## Get FindAllMarkers parameters

In [32]:
! grep --line-number -E -A 1 '^tiss.+FindAllMarkers' $rmds



--
Bladder/MACA_Plate_Notebook.new3mo.Rmd:275:tiss.markers <- FindAllMarkers(object = tiss, only.pos = TRUE, min.pct = 0.25, thresh.use = 0.25)
Bladder/MACA_Plate_Notebook.new3mo.Rmd-276-```
--
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd:182:tiss.markers <- FindAllMarkers(object = tiss, only.pos = TRUE, min.pct = 0.25, thresh.use = 0.25)
Brain_FACS_neurons/MACA_Plate_Notebook.Rmd-183-top_genes <- tiss.markers %>% group_by(cluster) %>% top_n(50, avg_diff)
--
Colon/MACA_Plate_Notebook_Colon_102717.Rmd:209:tiss.markers <- FindAllMarkers(object = tiss, only.pos = TRUE, min.pct = 0.25, thresh.use = 0.25)
Colon/MACA_Plate_Notebook_Colon_102717.Rmd-210-```
--
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd:222:tiss.markers <- FindAllMarkers(object = tiss, only.pos = TRUE, min.pct = 0.25, 
Diaphragm/MACA_Plate_Notebook_DiaSubtissue.Rmd-223-    thresh.use = 0.25)
--


--
Heart/MACA_Plate_Notebook_HeartSample.Rmd:206:tiss.markers <- FindAllMarkers(object = tiss, only.pos = TRUE, min.pct = 0.25, 

All used the same parameters, `only.pos = TRUE, min.pct = 0.25, thresh.use = 0.25`

In [33]:
# parameters['FindAllMarkers_min.pct'] = 0.25
# parameters['FindAllMarkers_thresh.use'] = 0.25

In [34]:
parameters.to_csv('/Users/olgabot/code/maca/metadata/seurat_parameters.csv')

In [35]:
parameters.shape

(19, 9)